In [ ]:
from pathlib import Path
import os

# === Configure your settings here ===
# - Set your project root path
# - We build the MCP server config inline using ROOT and API_DOMAIN
# - Optionally paste your OpenAI API key (or leave empty to use environment)

ROOT = Path("/Users/eloh/egain-mcp")  # <- change to your local path
API_DOMAIN = "api.aidev.egain.cloud"         # <- change to your API domain (e.g., api.aidev.egain.cloud)

# OpenAI
OPENAI_API_KEY = ""                    # <- paste here or set env var OPENAI_API_KEY
OPENAI_MODEL = "gpt-4o-mini"           # <- change model if desired

# Apply environment variables for downstream cells
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print(f"ROOT set to: {ROOT}")
print("API_DOMAIN:", API_DOMAIN)
print("OPENAI_MODEL:", OPENAI_MODEL)


ROOT set to: /Users/eloh/egain-mcp
API_DOMAIN: api.aidev.egain.cloud
OPENAI_MODEL: gpt-4o-mini


# Build Your Own MCP Client (JSON config driven)

A friendly, step-by-step guide to connect from Jupyter to your Node MCP server using a JSON config (Claude/Cursor-style `mcpServers`). PLEASE PROVIDE A FREE OPENAI TOKEN TO TEST, SEE CODE BELOW.

### What you’ll do
- Install the Python MCP client
- Load server settings from an `.json` MCP config
- Start the server via stdio
- List available tools
- Optionally call a tool

### Works with configs like
```json
{
  "mcpServers": {
    "egain-mcp": {
      "command": "node",
      "args": [
        "/Users/eloh/test-knowledge-mcp-typescript/bin/mcp-server.js",
        "start",
        "--api-domain",
        "api.aidev.egain.cloud"
      ]
    }
  }
}
```

If you use Claude or Cursor, this will auto-detect `~/.cursor/mcp.json` or `~/Library/Application Support/Claude/claude_desktop_config.json`. You can also point to any custom JSON path.


## Step 1: Install requirements

We’ll use the official Python MCP client. Run this once (restart kernel if imports fail):


In [15]:
# Install and import dependencies
%pip install -U "mcp==1.19.0" anyio

import json
import os
from typing import Dict, Any

try:
    from mcp import StdioServerParameters
    from mcp.client.stdio import stdio_client
    from mcp.client.session import ClientSession
    print("✅ MCP imports ready")
except Exception as e:
    print("❌ MCP import failed. Ensure 'mcp' is installed and kernel is restarted if needed.")
    raise



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ MCP imports ready


In [16]:
from pathlib import Path
import shutil

# Respect ROOT from the first cell if defined
ROOT = globals().get("ROOT", Path.cwd().parent)
HELP = ROOT / "help"
src = ROOT / ".env"
dst = HELP / ".env"

shutil.copy(src, dst)
print(f"Copied {src} -> {dst}")

Copied /Users/eloh/egain-mcp/.env -> /Users/eloh/egain-mcp/help/.env


## Step 2: Build your MCP server config inline

We construct the MCP server settings right here using `ROOT` and `API_DOMAIN` from the first cell. No external JSON files are needed.

Equivalent JSON shape (for reference only):
```json
{
  "mcpServers": {
    "EgainMcp": {
      "command": "node",
      "args": ["/ABSOLUTE/PATH/bin/mcp-server.js", "start", "--api-domain", "api.example.com"]
    }
  }
}
```


In [17]:
# Build MCP server settings inline using ROOT and API_DOMAIN (from the first cell)
from pathlib import Path

ROOT = globals().get("ROOT", Path.cwd().parent)
API_DOMAIN = globals().get("API_DOMAIN", "api.example.com")

server_cfg = {
    "command": "node",
    "args": [
        str(ROOT / "bin" / "mcp-server.js"),
        "start",
        "--api-domain",
        API_DOMAIN,
    ],
    "env": {},
}

print("Using inline MCP server config:")
print(json.dumps(server_cfg, indent=2))


Using inline MCP server config:
{
  "command": "node",
  "args": [
    "/Users/eloh/egain-mcp/bin/mcp-server.js",
    "start",
    "--api-domain",
    "api.aidev.egain.cloud"
  ],
  "env": {}
}


In [18]:
from pathlib import Path
import os, json

ROOT = globals().get("ROOT", Path.cwd().parent)
NB = ROOT / "help"
ESM = ROOT / "esm"

CANDIDATES = [
    NB / ".env",
    ROOT / ".env",
    ESM / ".env",
    ESM / "src" / ".env",
]

print("\n🔎 .env discovery:")
for p in CANDIDATES:
    print(f"- {p}: {'FOUND' if p.exists() else 'missing'}")

print("\n🔎 Bearer token files:")
for p in [ROOT / ".bearer_token", ROOT / ".bearer_token_metadata"]:
    print(f"- {p}: {'FOUND' if p.exists() else 'missing'}")

# Inspect any --env flags passed into server args (if server_cfg is defined)
try:
    args = server_cfg.get("args", [])
    env_flags = []
    for i, v in enumerate(args):
        if v == "--env" and i + 1 < len(args):
            env_flags.append(args[i + 1])
    print("\n🔎 --env flags in server args:")
    if env_flags:
        for item in env_flags:
            print(f"- {item}")
    else:
        print("- none")
except NameError:
    print("\nℹ️ server_cfg not defined yet; run Step 2/3 first.")



🔎 .env discovery:
- /Users/eloh/egain-mcp/help/.env: FOUND
- /Users/eloh/egain-mcp/.env: FOUND
- /Users/eloh/egain-mcp/esm/.env: missing
- /Users/eloh/egain-mcp/esm/src/.env: missing

🔎 Bearer token files:
- /Users/eloh/egain-mcp/.bearer_token: FOUND
- /Users/eloh/egain-mcp/.bearer_token_metadata: FOUND

🔎 --env flags in server args:
- none


## Step 3: Connect to the MCP server and list tools

Connect to your MCP server via stdio using the config above, then enumerate the available tools to verify connectivity.


In [19]:
# Connect to the MCP server (stdio) and list tools

async def list_tools_from_config(server: Dict[str, Any]):
    params = StdioServerParameters(
        command=server["command"],
        args=server.get("args", []),
        env=server.get("env", None),
    )
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools_result = await session.list_tools()
            tools = tools_result.tools
            print(f"Found {len(tools)} tools:")
            for t in tools:
                print(f"- {t.name}")
            return tools

# Run it
await list_tools_from_config(server_cfg)


Found 8 tools:
- get-article
- get-announcements
- get-popular-articles
- get-portals
- make-suggestion
- query-answers
- query-search
- query-retrieve


[Tool(name='get-article', title=None, description='Get Article by ID\n\nGet Article by ID\n\n## Prerequisites\n- Requires a valid portal ID. If you don\'t have the portal ID, first call \'get-portals\' to get available portals.\n- Portal ID format: 2-4 letter prefix + dash + 4-15 digits (e.g., "EB-123456789")\n- Requires a valid article ID. Article ID format: 2-4 letter prefix + dash + 4-15 digits (e.g., "PROD-2996")\n\n## Overview\nThe Get Article by ID API allows a user to retrieve a specific article using its ID. Additional article attributes and contextual views can be specified in the query parameters.\n\nThis API returns structured authoring attributes of Issue, Environment, Cause and Confidence Level when the following conditions are met:\n- The "Allow Structured Authoring" setting is enabled at the partition/department level\n- The "Use Structured Authoring" flag is set on the article type\n', inputSchema={'type': 'object', 'properties': {'request': {'type': 'object', 'properti

## Step 4: Ask in natural language (OpenAI function calling)

Use OpenAI to interpret a prompt and trigger the appropriate MCP tool. Example: “get the popular articles in portal PZ-9999”. Set OPENAI_API_KEY in your environment.


In [20]:
# Requires OPENAI_API_KEY in your environment
# %pip install -U openai

import os, json
from typing import Dict, Any, List

try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("Please install OpenAI: pip install openai")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    raise RuntimeError("Set OPENAI_API_KEY in the first cell or export OPENAI_API_KEY.")

client = OpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = (
    "You convert user requests into MCP tool calls. "
    "Choose the best matching function from the provided list and extract arguments from the user's text. "
    "Return exactly one function call with only the required arguments, plus obvious optional ones when specified."
)

async def _discover_openai_functions(server: Dict[str, Any]) -> List[dict]:
    """Discover MCP tools at runtime and convert to OpenAI function schemas."""
    params = StdioServerParameters(
        command=server["command"],
        args=server.get("args", []),
        env=server.get("env", None),
    )
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools_result = await session.list_tools()
            functions: List[dict] = []
            for t in tools_result.tools:
                name = getattr(t, "name", None)
                desc = getattr(t, "description", "") or ""
                schema = getattr(t, "inputSchema", None)
                params_schema = schema if isinstance(schema, dict) and schema.get("type") == "object" else {"type": "object", "properties": {}}
                functions.append({"name": name, "description": desc, "parameters": params_schema})
            return functions

In [21]:
from typing import Dict, Any

async def _call_mcp_direct(server: Dict[str, Any], tool_name: str, arguments: Dict[str, Any]):
    params = StdioServerParameters(
        command=server["command"],
        args=server.get("args", []),
        env=server.get("env", None),
    )
    payload: Dict[str, Any]
    if isinstance(arguments, dict) and "request" in arguments:
        payload = arguments
    else:
        payload = {"request": arguments if isinstance(arguments, dict) else {}}
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, payload)
            if getattr(result, "content", None):
                item = result.content[0]
                data = getattr(item, "text", None) or getattr(item, "data", None) or str(item)
                return data
            return result

In [22]:
# Convert MCP JSON output to natural language using OpenAI
# Provides a wrapper that returns a concise, human-readable answer.

from typing import Any


def _stringify_for_llm(obj: Any, max_chars: int = 12000) -> str:
    try:
        s = obj if isinstance(obj, str) else json.dumps(obj, ensure_ascii=False, indent=2)
    except Exception:
        s = str(obj)
    if len(s) > max_chars:
        return s[:max_chars] + "\n...[truncated]..."
    return s


def _mcp_to_natural_language(user_text: str, raw_result: Any) -> str:
    raw_str = _stringify_for_llm(raw_result)
    comp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Rewrite API/JSON outputs into a concise, clear natural-language answer. "
                    "Do not output JSON. Prefer short paragraphs and bullets when helpful."
                ),
            },
            {
                "role": "user",
                "content": (
                    "User request:\n" + user_text +
                    "\n\nTool output (JSON-like):\n" + raw_str +
                    "\n\nWrite the final answer in natural language for the user."
                ),
            },
        ],
    )
    return comp.choices[0].message.content


async def run_nl_command_natural(server: Dict[str, Any], user_text: str):
    """Natural language → tool selection → MCP call → natural-language answer."""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_text},
        ],
        functions = await _discover_openai_functions(server_cfg),
        function_call="auto",
    )

    msg = resp.choices[0].message
    if getattr(msg, "function_call", None):
        fn_name = msg.function_call.name
        try:
            fn_args = json.loads(msg.function_call.arguments or "{}")
        except Exception:
            fn_args = {}
        print(f"🔧 LLM chose: {fn_name} with args: {fn_args}")
        result = await _call_mcp_direct(server, fn_name, fn_args)
        print("\n✅ MCP result (raw):\n", result)
        nl = _mcp_to_natural_language(user_text, result)
        print("\n💬 Natural language answer:\n", nl)
        return nl
    else:
        print("❌ LLM did not return a function call. Prompt: ", user_text)
        return None

# Example (uncomment to run):
await run_nl_command_natural(server_cfg, "get the popular articles in portal PZ-9999")



🔧 LLM chose: get-popular-articles with args: {'request': {'portalID': 'PZ-9999'}}

✅ MCP result (raw):
 {"ArticleResults":{"articles":[{"articleType":{"articleCategoryId":0,"articleTypeId":"999900000002020","typeName":"General","useStructuredAuthoring":false},"createdBy":{"firstName":"testauthor1","id":"1006","lastName":"testauthor1","userName":"testauthor1 testauthor1"},"createdDate":"2025-10-13T23:11:06.000Z","hasAttachments":false,"id":"PROD-2147","languageCode":"en-US","link":{"href":"/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2147?%24lang=en-US","rel":"self"},"modifiedBy":{"firstName":"testauthor1","id":"1006","lastName":"testauthor1","userName":"testauthor1 testauthor1"},"modifiedDate":"2025-10-13T23:11:06.000Z","name":" You qualify for 85% loan-to-value ratio with principal cap at $100,000, estimated APR spanning 7.5-11.5%, with monthly dues determined by loan size.","versionId":"PROD-1200"},{"articleType":{"articleCategoryId":0,"articleTypeId":"999900000002020","type

"Here are some popular articles from portal PZ-9999:\n\n1. **You qualify for 85% loan-to-value ratio with principal cap at $100,000**  \n   [Read more](https://www.example.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2147)\n\n2. **You're approved for 95% loan-to-value ratio with a maximum principal of $50,000**  \n   [Read more](https://www.example.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2145)\n\n3. **You're eligible for 90% loan-to-value ratio with a maximum of $75,000**  \n   [Read more](https://www.example.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2146)\n\n4. **You're pre-qualified for an 80% loan-to-value ratio with a maximum borrowing limit of $500,000**  \n   [Read more](https://www.example.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2148)\n\n5. **A Guide to Understanding Credit Cards**  \n   [Read more](https://www.example.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2135)\n\n6. **AI in Treasury Operations**  \